In [ ]:
# Author: Roshan Ram
# AndrewID: rram

import yfinance as yfinance

# import module_manager
# module_manager.review()

import yfinance as yf # to pull stock data with yf.download(name, yyyy-mm-dd of opening, yyyy-mm-dd of opening)

import numpy as np # used for everything lol
import pandas as pd # data mainpulation
import matplotlib.pyplot as plt # graphing/plotting
from keras import *

import time, random, copy

#####
import numpy as np
import os
import sys
import time
import pandas as pd 
# from tqdm._tqdm_notebook import tqdm_notebook
import pickle
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import optimizers
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import logging
####

# %matplotlib inline 
#just to make stuff look nice

In [ ]:
# stock = None
# while not isinstance(stock, str):
#     stock = input("Enter your desired stock. Only alphanumeric characters please.")
# openingInp = input("Enter your desired opening date. (yyyy-mm-dd)") #'2016-01-01'
# closingInp = input("Enter your desired closing date. (yyyy-mm-dd)") # '2019-08-01'


# data = yf.download(stock, openingInp, closingInp)


# CONSTANTS: ####################################################

TIME_STEPS = 10 # [rram]
batchSize = 32 # [rram]

# CONSTANTS #####################################################

def getData_LSTM(app):
    stock = None
#     while not isinstance(stock, str):
    stock = app.getUserInput("Enter your desired stock. Only alphanumeric characters please.")
    openingInp = app.getUserInput("Enter your desired opening date. (yyyy-mm-dd)") #'2016-01-01'
    closingInp = app.getUserInput("Enter your desired closing date. (yyyy-mm-dd)") # '2019-08-01'


    data = yf.download(stock, openingInp, closingInp)
    return data, stock 

# data, stock = getData(app)

In [ ]:
def createTimesteps(mat, y_col_index):
    # y_col_index is the index of column that would act as output column
    # total number of time-series samples would be len(mat) - TIME_STEPS
    dim_0 = mat.shape[0] - TIME_STEPS
    dim_1 = mat.shape[1]
    x = np.zeros((dim_0, TIME_STEPS, dim_1))
    y = np.zeros((dim_0,))

    for i in (range(dim_0)):
        x[i] = mat[i:TIME_STEPS+i]
        y[i] = mat[TIME_STEPS+i, y_col_index]
    print("length of time-series i/o",x.shape,y.shape)
    return x, y

def createDataset(mat, batch_size):
    """
    trims dataset to a size that's divisible by batchSize
    """
    no_of_rows_drop = mat.shape[0]%batch_size
    if(no_of_rows_drop > 0):
        return mat[:-no_of_rows_drop]
    else:
        return mat

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

def mainFunc(data):

    #####
    params = {
        "batch_size": 20,  # 20<16<10, 25 was a bust
        "epochs": 300,
        "lr": 0.00010000,
        "time_steps": 60
    }

#     iter_changes = "dropout_layers_0.4_0.4"

    # INPUT_PATH = PATH_TO_DRIVE_ML_DATA+"/inputs"
    OUTPUT_PATH = '~/Desktop'
    TIME_STEPS = params["time_steps"]
    batchSize = params["batch_size"]
    stime = time.time()
    ###


    train_cols = ["Open","High","Low","Close","Volume"]
    df_train, df_test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=False)
    print("Train and Test size", len(df_train), len(df_test))
    # scale the feature MinMax, build array
    x = df_train.loc[:,train_cols].values
    min_max_scaler = MinMaxScaler()
    x_train = min_max_scaler.fit_transform(x)
    x_test = min_max_scaler.transform(df_test.loc[:,train_cols])
    
    ####
        
#     x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))   # TESTING 8:07 PM
        
    x_t, y_t = createTimesteps(x_train, 3)
    print('x_t:', x_t, '\n', 'y_t:', y_t)
    x_t = createDataset(x_t, batchSize)
    y_t = createDataset(y_t, batchSize)
    x_temp, y_temp = createTimesteps(x_test, 3)
    x_val, x_test_t = np.split(createDataset(x_temp, batchSize),2)
    y_val, y_test_t = np.split(createDataset(y_temp, batchSize),2)
    
    # def create_model():
    lstm_model = Sequential()
    lstm_model.add(LSTM(100, batch_input_shape=(batchSize, TIME_STEPS, x_t.shape[2]), dropout=0.0, recurrent_dropout=0.0, stateful=True,     kernel_initializer='random_uniform'))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(20,activation='relu'))
    lstm_model.add(Dense(1,activation='sigmoid'))
    optimizer = optimizers.RMSprop(lr=0.00010000)
    lstm_model.compile(loss='mean_squared_error', optimizer=optimizer)
    
    model = lstm_model
    
    
#     csv_logger = CSVLogger(os.path.join(OUTPUT_PATH, 'your_log_name' + '.log'), append=True)

    # epochs = params['epochs']


    # NOTE TO SELF CHANGE EPOCHS BACK to 300
    history = model.fit(x_t, y_t, epochs= 5, verbose=2, batch_size=batchSize, 
                        shuffle=False, validation_data=(createDataset(x_val, batchSize), 
                        createDataset(y_val, batchSize))) #callbacks=[csv_logger])

    # history = model.fit(x_t, y_t, epochs=params["epochs"], verbose=2, batch_size=batchSize,
    #                         shuffle=False, validation_data=(createDataset(x_val, batchSize),
    #                         createDataset(y_val, batchSize)), callbacks=[es, mcp, csv_logger])
    return history, model, batchSize, x_test_t, y_test_t 

In [ ]:
# update: SAVED MODEL!!! IMPORTANT WILL SAVE TIME ON FRONTEND FOR USER [12/4/19]

# commented out model saving [12/4/19 7:50 PM ] bc it was throwing some werid error

# fileName = 'savedLSTM.h5'
# model.save(fileName)
# print(f"Saved model `{fileName}` to disk")
#######################################################################################################





def plotLoss(history, model, x_test_t, y_test_t):
    y_pred = model.predict(createDataset(x_test_t, batchSize), batch_size=batchSize)
    y_pred = y_pred.flatten()
    y_test_t = createDataset(y_test_t, batchSize)
    error = mean_squared_error(y_test_t, y_pred)
    print("Error is", error, y_pred.shape, y_test_t.shape)
    print(y_pred[0:15])
    print(y_test_t[0:15])

    # convert the predicted value to range of real data
    y_pred_org = (y_pred * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3]
    # min_max_scaler.inverse_transform(y_pred)
    y_test_t_org = (y_test_t * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3]
    # min_max_scaler.inverse_transform(y_test_t)
    print(y_pred_org[0:15])
    print(y_test_t_org[0:15])

    # Visualize the training data
    from matplotlib import pyplot as plt
    plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    #plt.show()

In [ ]:
# x_test_t

In [ ]:
# y_pred = lstm_model.predict(createDataset(x_test_t, batchSize), batch_size=batchSize)
# y_pred = y_pred.flatten()
# y_test_t = createDataset(y_test_t, batchSize)
# error = mean_squared_error(y_test_t, y_pred)
# print("Error is", error, y_pred.shape, y_test_t.shape)
# print(y_pred[0:15])
# print(y_test_t[0:15])
# y_pred_org = (y_pred * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3] # min_max_scaler.inverse_transform(y_pred)
# y_test_t_org = (y_test_t * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3] # min_max_scaler.inverse_transform(y_test_t)
# print(y_pred_org[0:15])
# print(y_test_t_org[0:15])

In [ ]:
# y_pred

In [ ]:
from matplotlib import pyplot as plt

def plotPrediction(x_test_t, y_test_t, batchSize):
    y_pred = lstm_model.predict(createDataset(x_test_t, batchSize), batch_size=batchSize)
    y_pred = y_pred.flatten()
    y_test_t = createDataset(y_test_t, batchSize)
    error = mean_squared_error(y_test_t, y_pred)
    print("Error is", error, y_pred.shape, y_test_t.shape)
    print(y_pred[0:15])
    print(y_test_t[0:15])
    y_pred_org = (y_pred * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3] # min_max_scaler.inverse_transform(y_pred)
    y_test_t_org = (y_test_t * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3] # min_max_scaler.inverse_transform(y_test_t)
    print(y_pred_org[0:15])
    print(y_test_t_org[0:15])

    plt.figure()
    plt.plot(y_pred_org)
    plt.plot(y_test_t_org)
    plt.title('Prediction vs Real Stock Price')
    plt.ylabel('Price')
    plt.xlabel('Days')
    plt.legend(['Prediction', 'Real'], loc='upper left')
    plt.show()
    print("program completed ", time.time() - stime)

In [ ]:
# x_test_t

In [ ]:
# try with real testing data

def showErrors(x_test_t, y_test_t, batchSize):
    y_pred = model.predict(createDataset(x_test_t, batchSize), batch_size=batchSize) #used to say lstm_model.predict
    y_pred = y_pred.flatten()
    y_test_t = createDataset(y_test_t, batchSize)
    error = mean_squared_error(y_test_t, y_pred)
    print("Error is", error, y_pred.shape, y_test_t.shape)
    print(y_pred[0:15])
    print(y_test_t[0:15])
    y_pred_org = (y_pred * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3] # min_max_scaler.inverse_transform(y_pred)
    y_test_t_org = (y_test_t * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3] # min_max_scaler.inverse_transform(y_test_t)
    print(y_pred_org[0:15])
    print(y_test_t_org[0:15])

In [ ]:
# is_update_model = True
# if model is None or is_update_model:
#     from keras import backend as K
#     print("Building model...")
#     print("checking if GPU available", K.tensorflow_backend._get_available_gpus())
#     model = create_model()
    
#     es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
#                        patience=40, min_delta=0.0001)
    
#     mcp = ModelCheckpoint(os.path.join(OUTPUT_PATH,
#                           "best_model.h5"), monitor='val_loss', verbose=1,
#                           save_best_only=True, save_weights_only=False, mode='min', period=1)

#     # Not used here. But leaving it here as a reminder for future
#     r_lr_plat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=30, 
#                                   verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
    
#     csv_logger = CSVLogger(os.path.join(OUTPUT_PATH, 'training_log_' + time.ctime().replace(" ","_") + '.log'), append=True)
    
#     history = model.fit(x_t, y_t, epochs=params["epochs"], verbose=2, batch_size=batchSize,
#                         shuffle=False, validation_data=(createDataset(x_val, batchSize),
#                         createDataset(y_val, batchSize)), callbacks=[es, mcp, csv_logger])
    
#     print("saving model...")
#     pickle.dump(model, open("lstm_model", "wb"))

# # model.evaluate(x_test_t, y_test_t, batch_size=batchSize
# y_pred = model.predict(createDataset(x_test_t, batchSize), batch_size=batchSize)
# y_pred = y_pred.flatten()
# y_test_t = createDataset(y_test_t, batchSize)
# error = mean_squared_error(y_test_t, y_pred)
# print("Error is", error, y_pred.shape, y_test_t.shape)
# print(y_pred[0:15])
# print(y_test_t[0:15])

# # convert the predicted value to range of real data
# y_pred_org = (y_pred * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3]
# # min_max_scaler.inverse_transform(y_pred)
# y_test_t_org = (y_test_t * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3]
# # min_max_scaler.inverse_transform(y_test_t)
# print(y_pred_org[0:15])
# print(y_test_t_org[0:15])

# # Visualize the training data
# from matplotlib import pyplot as plt
# plt.figure()
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# #plt.show()
# plt.savefig(os.path.join(OUTPUT_PATH, 'train_vis_BS_'+str(batchSize)+"_"+time.ctime()+'.png'))

# # load the saved best model from above
# saved_model = load_model(os.path.join(OUTPUT_PATH, 'best_model.h5')) # , "lstm_best_7-3-19_12AM",
# print(saved_model)

# y_pred = saved_model.predict(createDataset(x_test_t, batchSize), batch_size=batchSize)
# y_pred = y_pred.flatten()
# y_test_t = createDataset(y_test_t, batchSize)
# error = mean_squared_error(y_test_t, y_pred)
# print("Error is", error, y_pred.shape, y_test_t.shape)
# print(y_pred[0:15])
# print(y_test_t[0:15])
# y_pred_org = (y_pred * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3] # min_max_scaler.inverse_transform(y_pred)
# y_test_t_org = (y_test_t * min_max_scaler.data_range_[3]) + min_max_scaler.data_min_[3] # min_max_scaler.inverse_transform(y_test_t)
# print(y_pred_org[0:15])
# print(y_test_t_org[0:15])

# # Visualize the prediction
# from matplotlib import pyplot as plt
# plt.figure()
# plt.plot(y_pred_org)
# plt.plot(y_test_t_org)
# plt.title('Prediction vs Real Stock Price')
# plt.ylabel('Price')
# plt.xlabel('Days')
# plt.legend(['Prediction', 'Real'], loc='upper left')
# #plt.show()
# plt.savefig(os.path.join(OUTPUT_PATH, 'pred_vs_real_BS'+str(batchSize)+"_"+time.ctime()+'.png'))
# print_time("program completed ", stime)